# データ加工と学習（画像検出＋画像分類用）


Amazon SageMakerのノートブックインスタンスで使用することを想定しています。(諸々必要なパッケージを入れれば他の環境でも使用可能)

# 学習データの合成

### パッケージのインストール

In [ ]:
!pip install tqdm imgaug mxnet

### 学習データの準備
学習に使用するラベルづけされた画像データの取得を行います。
期待するデータ構造 は次の通りです。
```
dir ------ dataset1 ---- 1.jpg  
        |                |---2.jpg  
        |                |__3.jpg                   
        | - dataset1.lst
        |- dataset2 ---- 1.jpg
        |                |---2.jpg
        |                |__3.jpg                   
        | - dataset2.lst
        ...
```

以下の処理は例です。必要に応じて変更してください。

In [ ]:
import boto3
s3 = boto3.resource('s3')

ex_bucket_name = 'bucket'
ex_object_key = 'hogeobj_key'
ex_object_name = 'hoge.zip'
s3.Bucket(ex_bucket_name).download_file(ex_object_key, ex_object_name)

In [ ]:
!unzip -d hoge hoge.zip

## データの合成処理の実行

In [ ]:
import imgmlutil
# lstファイルのパス
lst_path_list = [
    './path/to/lst/1.lst',
    './path/to/lst/2.lst',
]

# 画像ファイルの位置(順序はlstファイルと対応づける)
img_root_path_list = [
    './path/to/img_dir/1',
    './path/to/img_dir/2',
]

# 読み込んだlstファイルをマージしたもの出力先ルート
merged_root_path = './data/merged'

# マージ処理実行
imgmlutil.merge_annotated_img(lst_path_list, img_root_path_list, merged_root_path)

# 画像増幅器の定義

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa

# シードを固定
ia.seed(1)

# 画像増幅のためのaugmentorを定義
aug_templates = [
    iaa.Invert(1, per_channel=0.5), #  各ピクセルの値を反転させる
    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.25)), #ところどころ欠落させる
    iaa.CoarseDropout((0.03, 0.15), size_percent=0.02, per_channel=0.8), # ところどころ色を変える
    iaa.CoarseSaltAndPepper(0.2, size_percent=(0.05, 0.1)), # 白と黒のノイズ
    iaa.WithChannels(0, iaa.Affine(rotate=(0,10))), # 赤い値を傾ける
    iaa.FrequencyNoiseAlpha( # 決まった形のノイズを加える
        first=iaa.EdgeDetect(1),
        per_channel=0.5
    ),
    iaa.ElasticTransformation(sigma=0.5, alpha=1.0), # モザイクをかける
    iaa.AddToHueAndSaturation(value=25), # 色調と彩度に値を追加
    iaa.Emboss(alpha=1.0, strength=1.5), # 浮き出し加工
    iaa.Superpixels(n_segments=100, p_replace=0.5), # superpixel表現にして、各セル内を一定確率でセルの平均値で上書きする
    iaa.Fliplr(1.0),
    iaa.Flipud(1.0)
]

# 画像増幅に使用するaugmentor
img_augmentors = [
    iaa.Noop(), # 無変換
    iaa.SomeOf(1, aug_templates),
    iaa.SomeOf(1, aug_templates),
    iaa.SomeOf(1, aug_templates),
    iaa.SomeOf(2, aug_templates),
    iaa.SomeOf(2, aug_templates),
    iaa.SomeOf(2, aug_templates),
    iaa.SomeOf(3, aug_templates),
]

# 画像分類用のデータ加工

### 加工処理

In [ ]:
from os import path
import imgmlutil

# 画像サイズ(変換後の画像サイズ: img_edge_size * img_edge_size)
img_edge_size = 224

# lstファイルの入力パス
input_lst_path = path.join(merged_root_path, 'lst.lst')

# 入力する画像が入っているディレクトリのパス
input_img_root_path = path.join(merged_root_path, 'img')

# 出力先
output_root_path = './data/auged/'

# 除外したいデータのクラス
except_class_list = ["4", "5", "6"]

# 処理実行(戻り値:trainとvalのそれぞれのデータ数)
cla_data_count = imgmlutil.process_image_for_classification(0.2, img_augmentors, img_edge_size, input_lst_path, input_img_root_path, output_root_path, except_class_list)


## RecordIO形式に加工して、S3にアップロード

In [ ]:
import imgmlutil

# 保存場所
bucket_name = "bucketname"
s3_classification_prefix = 'test/classification'
cla_s3_train_rec = path.join(s3_classification_prefix, 'train.rec')
cla_s3_val_rec = path.join(s3_classification_prefix, 'val.rec')

# 処理実行(学習用と検証用)
imgmlutil.create_recordio_and_upload_to_s3( path.join(output_root_path, 'train'), bucket_name, cla_s3_train_rec)
imgmlutil.create_recordio_and_upload_to_s3( path.join(output_root_path, 'val'), bucket_name, cla_s3_val_rec)

# 検出器用のデータ加工

### 加工処理

In [ ]:
from os import path
import imgmlutil

# subに入っているクラスのラベルを該当するlabel_idに変換するためのマップデータ
class_map=[{
    'label_id': 1,
    'label': 'bird',
    'sub': {
        '1' : 'duck',
        '2' : 'swallow',
        '3' : 'owl'}
},{
    'label_id': 2,
    'label': 'dog',
    'sub': {'9' : 'bulldog', '10':'Chihuahua'}
},{
    'label_id': 3,
    'label': 'cat',
    'sub': {'8' : 'savannah'}
}]

# 入出力先定義
input_lst_path = path.join(merged_root_path, 'lst.lst')
input_img_root_path = path.join(merged_root_path, 'img')
output_root_path = './data/detection_auged/'

# 変換したい画像サイズ（img_edg_size * img_edge_sizeの正方形に変換する)
img_edge_size = 512

# 処理を実行(戻り値:trainとvalのそれぞれのデータ数)
det_data_count = imgmlutil.process_image_for_detection(0.2, img_augmentors, img_edge_size, input_lst_path, input_img_root_path, output_root_path, class_map)

## RecordIO形式に加工して、S3にアップロード

In [ ]:
import imgmlutil

# 保存場所
bucket_name = "bucketname"
s3_detection_prefix = 'test/detection'
det_s3_train_rec = path.join(s3_detection_prefix, 'train.rec')
det_s3_val_rec = path.join(s3_detection_prefix, 'val.rec')


# 処理実行(学習用と検証用)
imgmlutil.create_recordio_and_upload_to_s3_from_lst(
    path.join(output_root_path, 'train'),
    path.join(output_root_path, 'train.lst'),
    bucket_name,
    det_s3_train_rec)

imgmlutil.create_recordio_and_upload_to_s3_from_lst( 
    path.join(output_root_path, 'val'),
    path.join(output_root_path, 'val.lst'),
    bucket_name,
    det_s3_val_rec)

# 学習


In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
sess = sagemaker.Session()

## 分類器

In [ ]:
import time
from time import gmtime, strftime
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# データの引き継ぎ
bucket_name = bucket_name
cla_s3_train_rec = cla_s3_train_rec
cla_s3_val_rec =  cla_s3_val_rec
s3_classification_prefix = s3_classification_prefix
train_data_count = cla_data_count['train']

# 入力データ定義
train_data = sagemaker.session.s3_input( 's3://'+path.join(bucket_name, cla_s3_train_rec), distribution='FullyReplicated', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input( 's3://'+path.join(bucket_name, cla_s3_val_rec), distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix')
data_channels = { 'train':train_data, 'validation':validation_data }

# モデルアーティファクトの出力先
s3_output_location = 's3://' + path.join(bucket_name, s3_classification_prefix, 'output', time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime()))


# 学習に使用するコンテナイメージ
training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

# 画像分類の学習時の設定
cl_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.8xlarge',
                                         train_volume_size = 20,
                                         train_max_run = 7200,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

# ハイパーパラメータの設定
cl_model.set_hyperparameters(image_shape='3,224,224',
                             num_layers=152,
                             use_pretrained_model=1,
                             num_classes=3,
                             mini_batch_size=32,
                             epochs=10,
                             learning_rate=0.001,
                             lr_scheduler_step=10,
                             top_k=3,
                             optimizer='sgd',
                             checkpoint_frequency=10,
                             momentum=0.9,
                             weight_decay=0.0005,
                             num_training_samples=train_data_count)

# ハイパーパラメータチューニングの探索範囲設定
hyperparameter_ranges  = {'mini_batch_size': IntegerParameter(16, 64),
                        'learning_rate': ContinuousParameter(1e-6, 0.5),
                        'optimizer': CategoricalParameter(['sgd', 'adam', 'rmsprop', 'nag']),
                        'momentum': ContinuousParameter(0, 0.999),
                        'weight_decay': ContinuousParameter(0, 0.999),
                        'beta_1': ContinuousParameter(1e-6, 0.999),
                        'beta_2': ContinuousParameter(1e-6, 0.999),
                        'eps': ContinuousParameter(1e-8, 1.0),
                        'gamma': ContinuousParameter(1e-8, 0.999)}

# ハイパーパラメータチューニングの目的関数
objective_metric_name = 'validation:accuracy'


# ハイパーパラメータチューニング用のチューナー定義
tuner = HyperparameterTuner(cl_model,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=2,
                            max_parallel_jobs=2)

# チューニング開始(稼働時間に対してお金がかかるので注意！)
tuner.fit(inputs=data_channels, logs=True, wait=False, include_cls_metadata=False)

## 検出器

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# データの引き継ぎ
bucket_name = bucket_name
det_s3_train_rec = det_s3_train_rec
det_s3_val_rec =  det_s3_val_rec
s3_detection_prefix = s3_detection_prefix
train_data_count =  det_data_count['train']


# データ
train_data = sagemaker.session.s3_input( 's3://'+path.join(bucket_name, det_s3_train_rec), distribution='FullyReplicated', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input( 's3://'+path.join(bucket_name, det_s3_val_rec), distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix')
data_channels = { 'train':train_data, 'validation':validation_data }

# モデルアーティファクトの出力先
s3_output_location = 's3://' + path.join(bucket_name, s3_detection_prefix, 'output', time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime()))


# 学習に使用するコンテナイメージ
training_image = get_image_uri(boto3.Session().region_name, 'object-detection')

# 物体検出用の学習時の設定
od_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         train_volume_size = 20,
                                         train_max_run = 7200,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

# ハイパーパラメータの設定
od_model.set_hyperparameters(base_network='resnet-50',
                             use_pretrained_model=1,
                             num_classes=3,
                             mini_batch_size=32,
                             epochs=10,
                             learning_rate=0.001,
                             lr_scheduler_step='10',
                             lr_scheduler_factor=0.1,
                             optimizer='sgd',
                             momentum=0.9,
                             weight_decay=0.0005,
                             overlap_threshold=0.5,
                             nms_threshold=0.45,
                             image_shape=512,
                             label_width=100,
                             num_training_samples=train_data_count)

# ハイパーパラメータチューニングの探索範囲設定
hyperparameter_ranges  = {'mini_batch_size': IntegerParameter(16, 32),
                        'learning_rate': ContinuousParameter(1e-6, 0.5),
                        'optimizer': CategoricalParameter(['sgd', 'adam', 'adadelta']),
                        'momentum': ContinuousParameter(0, 0.999),
                        'weight_decay': ContinuousParameter(0, 0.999)}

# ハイパーパラメータチューニングの目的関数
objective_metric_name = 'validation:mAP'

# ハイパーパラメータチューニング用のチューナー定義
tuner = HyperparameterTuner(od_model,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=2,
                            max_parallel_jobs=2)

# チューニング開始(稼働時間に対してお金がかかるので注意！)
tuner.fit(inputs=data_channels, logs=True, wait=False)